In [ ]:
# Databricks notebook source
# =============================================================================
# 🎯 IMPROVED MODEL REGISTRATION - BEST RUN SELECTION (MLflow + UC)
# =============================================================================
# Enhanced with better error handling, validation, and flexibility
# =============================================================================

import mlflow
from mlflow.tracking import MlflowClient
import sys
import os
import requests
import traceback
from typing import Dict, Optional, Tuple, Any
from datetime import datetime

# ====================== CONFIGURATION ========================= #
class Config:
    """Centralized configuration management"""
    
    def __init__(self):
        self.EXPERIMENT_NAME = self._get_experiment_name()
        self.UC_CATALOG = "workspace"
        self.UC_SCHEMA = "ml"
        self.SLACK_WEBHOOK_URL = self._get_slack_webhook()
        self.TOL = 1e-6  # Float comparison tolerance
        
        # Model configurations
        self.MODEL_CONFIG = {
            "xgboost": {
                "model_name": "house_price_xgboost_uc2",
                "artifact_path": "xgboost_model",
                "param_keys": [
                    "n_estimators", "max_depth", "learning_rate",
                    "subsample", "colsample_bytree"
                ],
                "metric_key": "test_rmse",
                "keywords": ["xgboost", "xgb", "house_price", "config_runs", "prediction"],
                "priority": 1
            },
            "randomforest": {
                "model_name": "house_price_rf_uc",
                "artifact_path": "rf_model",
                "param_keys": [
                    "n_estimators", "max_depth", "min_samples_split"
                ],
                "metric_key": "test_rmse",
                "keywords": ["randomforest", "rf", "random_forest"],
                "priority": 2
            }
        }
        
    def _get_experiment_name(self) -> str:
        """Get experiment name from widget or use default"""
        try:
            widget_value = dbutils.widgets.get("experiment_name")
            if widget_value and widget_value.strip():
                print(f"✓ Experiment Name from widget: {widget_value}")
                return widget_value.strip()
        except Exception as e:
            print(f"ℹ Widget not available or error: {e}")
        
        default_exp = "/Shared/House_Price_Prediction_Config_Runs"
        print(f"ℹ Using default experiment: {default_exp}")
        return default_exp
    
    def _get_slack_webhook(self) -> Optional[str]:
        """Safely retrieve Slack webhook URL with fallback to dev-scope"""
        for scope in ["shared-scope", "dev-scope"]:
            try:
                webhook = dbutils.secrets.get(scope, "SLACK_WEBHOOK_URL")
                if webhook and webhook.strip():
                    print(f"✓ Slack webhook configured from scope '{scope}'")
                    return webhook
            except Exception as e:
                print(f"⚠️ Slack webhook not found in scope '{scope}': {e}")
        return None


# Initialize configuration
config = Config()

# =================== SLACK NOTIFICATION HELPER ==================== #
class SlackNotifier:
    """Enhanced Slack notification handler"""
    
    def __init__(self, webhook_url: Optional[str]):
        self.webhook_url = webhook_url
        self.enabled = webhook_url is not None
        
    def send(self, message: str, level: str = "info") -> bool:
        """
        Send Slack notification with error handling
        
        Args:
            message: Message to send
            level: Notification level (info, success, warning, error)
        """
        if not self.enabled:
            print(f"📢 [SLACK DISABLED] {message}")
            return False
            
        emoji_map = {
            "info": "ℹ️",
            "success": "✅",
            "warning": "⚠️",
            "error": "❌"
        }
        
        formatted_message = f"{emoji_map.get(level, 'ℹ️')} {message}"
        payload = {"text": formatted_message}
        
        try:
            response = requests.post(
                self.webhook_url, 
                json=payload,
                timeout=5  # Add timeout
            )
            if response.status_code == 200:
                print(f"📢 Slack notification sent: {level}")
                return True
            else:
                print(f"⚠️ Slack error: {response.status_code} - {response.text}")
                return False
        except Exception as e:
            print(f"❌ Slack notification failed: {e}")
            return False


# Initialize Slack notifier
slack = SlackNotifier(config.SLACK_WEBHOOK_URL)

# ================== MODEL TYPE DETECTION ====================== #
class ModelDetector:
    """Intelligent model type detection"""
    
    def __init__(self, model_config: Dict):
        self.model_config = model_config
        
    def detect(self, experiment_name: str) -> Tuple[str, str, list, str]:
        """
        Detect model configuration from experiment name
        
        Returns:
            (full_uc_name, artifact_path, param_keys, metric_key)
        """
        exp_lower = experiment_name.lower()
        
        print(f"\n{'='*70}")
        print("🔍 MODEL TYPE DETECTION")
        print(f"{'='*70}")
        print(f"Experiment: {experiment_name}")
        
        # Sort by priority
        sorted_configs = sorted(
            self.model_config.items(),
            key=lambda x: x[1].get("priority", 999)
        )
        
        # Try to match keywords
        for model_type, cfg in sorted_configs:
            for keyword in cfg["keywords"]:
                if keyword in exp_lower:
                    full_uc_name = f"{config.UC_CATALOG}.{config.UC_SCHEMA}.{cfg['model_name']}"
                    print(f"✓ Detected model type: {model_type.upper()}")
                    print(f"✓ UC Model Name: {full_uc_name}")
                    print(f"✓ Artifact Path: {cfg['artifact_path']}")
                    print(f"✓ Metric: {cfg['metric_key']}")
                    print(f"{'='*70}\n")
                    
                    return (
                        full_uc_name,
                        cfg["artifact_path"],
                        cfg["param_keys"],
                        cfg["metric_key"]
                    )
        
        # Fallback to first (highest priority) config
        default_type, default_cfg = sorted_configs[0]
        full_uc_name = f"{config.UC_CATALOG}.{config.UC_SCHEMA}.{default_cfg['model_name']}"
        
        print(f"⚠️ No exact match found")
        print(f"✓ Using default: {default_type.upper()}")
        print(f"✓ UC Model Name: {full_uc_name}")
        print(f"{'='*70}\n")
        
        return (
            full_uc_name,
            default_cfg["artifact_path"],
            default_cfg["param_keys"],
            default_cfg["metric_key"]
        )


# Initialize detector
detector = ModelDetector(config.MODEL_CONFIG)
REGISTERED_MODEL_NAME, ARTIFACT_PATH, PARAM_KEYS, METRIC_KEY = detector.detect(config.EXPERIMENT_NAME)

# ====================== UTILITIES ====================== #
def normalize(val: Any) -> Any:
    """Normalize parameter values for comparison"""
    try:
        val_str = str(val)
        # Check if it's an integer
        if '.' not in val_str and val_str.lstrip('-').isdigit():
            return int(val)
        # Try float conversion
        return float(val)
    except (ValueError, TypeError):
        return str(val)


def validate_experiment(client: MlflowClient, exp_name: str) -> bool:
    """Validate that experiment exists and has runs"""
    try:
        exp = client.get_experiment_by_name(exp_name)
        if not exp:
            print(f"\n❌ ERROR: Experiment '{exp_name}' not found!")
            print("\n💡 Available experiments:")
            all_exps = client.search_experiments(max_results=10)
            for e in all_exps:
                print(f"   - {e.name}")
            return False
        return True
    except Exception as e:
        print(f"❌ Experiment validation failed: {e}")
        return False


# ================== FIND BEST RUN ====================== #
def get_best_run(client: MlflowClient) -> Tuple[Optional[str], Dict, Dict]:
    """
    Find the best run based on metric optimization
    
    Returns:
        (run_id, params, metrics) or (None, {}, {})
    """
    print(f"\n{'='*70}")
    print("🔍 SEARCHING FOR BEST RUN")
    print(f"{'='*70}")
    
    # Validate experiment exists
    if not validate_experiment(client, config.EXPERIMENT_NAME):
        return None, {}, {}
    
    try:
        exp = client.get_experiment_by_name(config.EXPERIMENT_NAME)
        
        # Search for runs with valid metrics
        runs = client.search_runs(
            [exp.experiment_id],
            filter_string=f"metrics.{METRIC_KEY} > 0",
            order_by=[f"metrics.{METRIC_KEY} ASC"],
            max_results=1000
        )
        
        if not runs:
            print(f"⚠️ No runs found with valid '{METRIC_KEY}' metric")
            return None, {}, {}
        
        print(f"✓ Found {len(runs)} runs with valid metrics\n")
        
        # Display top runs for transparency
        print(f"📊 Top 10 Runs (by {METRIC_KEY}):")
        print(f"{'Rank':<6} {'Run Name':<35} {'Metric':<12} {'Timestamp':<20}")
        print("-" * 90)
        
        for i, run in enumerate(runs[:10], 1):
            run_name = run.info.run_name or "Unnamed"
            metric_val = run.data.metrics.get(METRIC_KEY, float('inf'))
            timestamp = datetime.fromtimestamp(run.info.start_time/1000).strftime('%Y-%m-%d %H:%M')
            marker = "👑" if i == 1 else f"{i}."
            print(f"{marker:<6} {run_name:<35} {metric_val:<12.6f} {timestamp}")
        
        # Best run is the first one
        best_run = runs[0]
        best_metric = best_run.data.metrics.get(METRIC_KEY)
        
        # Extract parameters
        params = {
            k: normalize(v) 
            for k, v in best_run.data.params.items() 
            if k in PARAM_KEYS
        }
        
        metrics = best_run.data.metrics
        
        print(f"\n{'='*70}")
        print("🏆 BEST RUN SELECTED")
        print(f"{'='*70}")
        print(f"   Run ID: {best_run.info.run_id}")
        print(f"   Run Name: {best_run.info.run_name or 'N/A'}")
        print(f"   {METRIC_KEY}: {best_metric:.6f}")
        print(f"   Rank: #1 out of {len(runs)} runs")
        print(f"   Parameters: {params}")
        print(f"{'='*70}\n")
        
        return best_run.info.run_id, params, metrics
        
    except Exception as e:
        print(f"❌ Error finding best run: {e}")
        traceback.print_exc()
        return None, {}, {}


# ================ DUPLICATE VERSION CHECK ===================== #
def check_duplicate(
    client: MlflowClient, 
    new_params: Dict, 
    new_metrics: Dict
) -> Optional[Any]:
    """
    Check if a model with same params and metrics already exists
    
    Returns:
        ModelVersion object if duplicate found, else None
    """
    print(f"\n{'='*70}")
    print("🔍 CHECKING FOR DUPLICATES")
    print(f"{'='*70}")
    
    try:
        mv_list = client.search_model_versions(
            f"name = '{REGISTERED_MODEL_NAME}'"
        )
    except Exception as e:
        print(f"ℹ️ No existing model versions (first registration): {e}")
        return None
    
    if not mv_list:
        print("ℹ️ No existing versions found")
        return None
    
    print(f"✓ Found {len(mv_list)} existing version(s)")
    
    new_metric_val = new_metrics.get(METRIC_KEY)
    if new_metric_val is None:
        print("⚠️ New model missing metric - cannot check duplicates")
        return None
    
    # Check each existing version
    for mv in mv_list:
        try:
            run = client.get_run(mv.run_id)
        except Exception as e:
            print(f"⚠️ Could not fetch run {mv.run_id}: {e}")
            continue
        
        # Compare parameters
        old_params = {
            k: normalize(v) 
            for k, v in run.data.params.items() 
            if k in new_params
        }
        
        old_metric_val = run.data.metrics.get(METRIC_KEY)
        
        # Check if parameters match
        same_params = all(
            old_params.get(k) == new_params.get(k) 
            for k in new_params
        )
        
        # Check if metrics match (within tolerance)
        same_metric = (
            old_metric_val is not None 
            and abs(old_metric_val - new_metric_val) <= config.TOL
        )
        
        if same_params and same_metric:
            print(f"\n⏭️ DUPLICATE DETECTED!")
            print(f"   Existing Version: v{mv.version}")
            print(f"   Run ID: {mv.run_id}")
            print(f"   Status: {mv.current_stage}")
            print(f"   {METRIC_KEY}: {old_metric_val:.6f}")
            print(f"   Params match: ✓")
            print(f"   Metric match: ✓")
            print(f"\n   → Skipping registration, using existing version")
            return mv
    
    print("✓ No duplicates found - proceeding with registration")
    return None


# ================== REGISTER MODEL LOGIC ======================= #
def register_model(
    client: MlflowClient, 
    run_id: str, 
    params: Dict, 
    metrics: Dict
) -> Optional[Any]:
    """
    Register model with comprehensive validation and tagging
    
    Returns:
        ModelVersion object if successful, None otherwise
    """
    # Check for duplicates first
    duplicate_version = check_duplicate(client, params, metrics)
    if duplicate_version:
        slack.send(
            f"⚠️ Duplicate detected — using existing model version *v{duplicate_version.version}* "
            f"for `{REGISTERED_MODEL_NAME}`",
            level="warning"
        )
        return duplicate_version
    
    # Proceed with registration
    model_uri = f"runs:/{run_id}/{ARTIFACT_PATH}"
    
    print(f"\n{'='*70}")
    print("⏳ REGISTERING NEW MODEL VERSION")
    print(f"{'='*70}")
    print(f"   Model URI: {model_uri}")
    print(f"   Target: {REGISTERED_MODEL_NAME}")
    
    try:
        # Register the model
        new_version = mlflow.register_model(model_uri, REGISTERED_MODEL_NAME)
        
        print(f"\n{'='*70}")
        print("✅ MODEL REGISTERED SUCCESSFULLY!")
        print(f"{'='*70}")
        print(f"   Model Name: {REGISTERED_MODEL_NAME}")
        print(f"   Version: v{new_version.version}")
        print(f"   Source Run ID: {run_id}")
        print(f"   {METRIC_KEY}: {metrics.get(METRIC_KEY, 'N/A')}")
        print(f"{'='*70}\n")
        
        # Add comprehensive tags
        tags_to_add = {
            "source_run_id": run_id,
            "experiment_name": config.EXPERIMENT_NAME,
            "metric_rmse": str(metrics.get(METRIC_KEY, "")),
            "registration_timestamp": datetime.now().isoformat(),
            "artifact_path": ARTIFACT_PATH
        }
        
        # Add parameter tags
        for param_key, param_val in params.items():
            tags_to_add[f"param_{param_key}"] = str(param_val)
        
        # Apply all tags
        print("🏷️  Adding tags...")
        for tag_key, tag_val in tags_to_add.items():
            try:
                client.set_model_version_tag(
                    REGISTERED_MODEL_NAME,
                    new_version.version,
                    tag_key,
                    tag_val
                )
                print(f"   ✓ {tag_key}: {tag_val}")
            except Exception as e:
                print(f"   ⚠️ Failed to add tag {tag_key}: {e}")
        
        # Send success notification
        slack.send(
            f"✅ Model *{REGISTERED_MODEL_NAME}* registered as version *v{new_version.version}*\n"
            f"🏷 Metric `{METRIC_KEY}` = {metrics.get(METRIC_KEY, 'N/A'):.6f}\n"
            f"🔗 Run ID: {run_id}",
            level="success"
        )
        
        return new_version
        
    except Exception as e:
        print(f"\n❌ REGISTRATION FAILED")
        print(f"   Error: {e}")
        traceback.print_exc()
        
        slack.send(
            f"❌ Model registration failed for `{REGISTERED_MODEL_NAME}` — {e}",
            level="error"
        )
        
        return None


# ============================ MAIN ============================ #
def main():
    """Main execution flow with comprehensive error handling"""
    print("\n" + "=" * 80)
    print("🚀 MODEL REGISTRATION - BEST RUN SELECTION (MLflow + UC)")
    print("=" * 80 + "\n")
    
    try:
        # Initialize MLflow client
        client = MlflowClient()
        
        # Display configuration
        print(f"📋 Configuration:")
        print(f"   Experiment: {config.EXPERIMENT_NAME}")
        print(f"   Target Model: {REGISTERED_MODEL_NAME}")
        print(f"   Metric: {METRIC_KEY} (lower is better)")
        print(f"   Artifact Path: {ARTIFACT_PATH}")
        print(f"   Slack Notifications: {'Enabled ✓' if slack.enabled else 'Disabled ✗'}")
        print()
        
        # Find best run
        print("🔍 Searching for best run...")
        run_id, params, metrics = get_best_run(client)
        
        if not run_id:
            print("\n❌ No valid best run found")
            print("\n💡 Troubleshooting:")
            print("   1. Verify training script completed successfully")
            print("   2. Check experiment name configuration")
            print("   3. Ensure metrics were logged during training")
            
            slack.send(
                f"⚠️ Model registration skipped — no valid best run found for `{REGISTERED_MODEL_NAME}`",
                level="warning"
            )
            sys.exit(1)
        
        # Register the model
        model_version = register_model(client, run_id, params, metrics)
        
        if model_version:
            print("\n✨ Registration process completed successfully!")
            print(f"   Version: v{model_version.version}")
            print(f"   Status: {model_version.current_stage or 'None'}")
            sys.exit(0)
        else:
            print("\n❌ Registration failed")
            sys.exit(1)
            
    except KeyboardInterrupt:
        print("\n\n⚠️ Process interrupted by user")
        sys.exit(1)
    except Exception as e:
        print(f"\n❌ UNEXPECTED ERROR: {e}")
        traceback.print_exc()
        
        slack.send(
            f"❌ Critical error in model registration: {e}",
            level="error"
        )
        sys.exit(1)


# ============================ ENTRY POINT ============================ #
if __name__ == "__main__":
    main()





# # Databricks notebook source
# import mlflow
# from mlflow.tracking import MlflowClient
# import sys
# import os
# import requests  # ✅ Added for Slack webhook notifications

# # ====================== CONFIGURATION ========================= #
# try:
#     EXPERIMENT_NAME = dbutils.widgets.get("experiment_name")
#     print(f"✓ Experiment Name from widget: {EXPERIMENT_NAME}")
# except:
#     EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Config_Runs"
#     print(f"ℹ Using default experiment: {EXPERIMENT_NAME}")

# UC_CATALOG = "workspace"
# UC_SCHEMA = "ml"

# # =================== MODEL CONFIG METADATA ==================== #
# MODEL_CONFIG = {
#     "xgboost": {
#         "model_name": "house_price_xgboost_uc2",
#         "artifact_path": "xgboost_model",
#         "param_keys": [
#             "n_estimators", "max_depth", "learning_rate",
#             "subsample", "colsample_bytree"
#         ],
#         "metric_key": "test_rmse",
#         "keywords": ["xgboost", "xgb", "house_price", "config_runs"]
#     }
# }

# # =================== SLACK NOTIFICATION HELPER ==================== #
# def send_slack_notification(message):
#     """
#     Send Slack message using webhook URL.
#     Set SLACK_WEBHOOK_URL as environment variable or Databricks secret.
#     """
#     webhook_url = os.getenv("SLACK_WEBHOOK_URL")
#     if not webhook_url:
#         print("⚠️ No Slack webhook URL found. Skipping notification.")
#         return

#     payload = {"text": message}
#     try:
#         response = requests.post(webhook_url, json=payload)
#         if response.status_code == 200:
#             print("📢 Slack notification sent successfully.")
#         else:
#             print(f"⚠️ Slack notification failed: {response.status_code}, {response.text}")
#     except Exception as e:
#         print(f"❌ Slack notification error: {e}")

# # ================== MODEL TYPE DETECTION ====================== #
# def detect_model_config(experiment_name: str):
#     exp_lower = experiment_name.lower()
#     for model_type, cfg in MODEL_CONFIG.items():
#         for key in cfg["keywords"]:
#             if key in exp_lower:
#                 full_uc_name = f"{UC_CATALOG}.{UC_SCHEMA}.{cfg['model_name']}"
#                 print(f"✓ Detected model type: {model_type.upper()}")
#                 print(f"✓ UC Model Name: {full_uc_name}")
#                 return (
#                     full_uc_name,
#                     cfg["artifact_path"],
#                     cfg["param_keys"],
#                     cfg["metric_key"]
#                 )
#     raise ValueError("❌ No matching model config found based on experiment name!")

# REGISTERED_MODEL_NAME, ARTIFACT_PATH, PARAM_KEYS, METRIC_KEY = detect_model_config(EXPERIMENT_NAME)
# TOL = 1e-6  # float tolerance

# # ====================== UTILITIES ====================== #
# def normalize(val):
#     try:
#         if '.' not in str(val) and str(val).isdigit():
#             return int(val)
#         return float(val)
#     except:
#         return str(val)

# # ================== FIND BEST RUN ====================== #
# def get_best_run(client):
#     exp = client.get_experiment_by_name(EXPERIMENT_NAME)
#     if not exp:
#         print("❌ Experiment not found.")
#         return None, {}, {}

#     runs = client.search_runs(
#         [exp.experiment_id], 
#         order_by=[f"metrics.{METRIC_KEY} ASC"],
#         max_results=1000
#     )
    
#     if not runs:
#         print("⚠ No runs found in experiment.")
#         return None, {}, {}

#     best_run = None
#     best_metric = float("inf")

#     for r in runs:
#         metric_val = r.data.metrics.get(METRIC_KEY)
#         if metric_val is not None:
#             print(f"  📊 Run: {r.info.run_name or r.info.run_id[:8]} | {METRIC_KEY}: {metric_val:.4f}")
#             if metric_val < best_metric:
#                 best_metric = metric_val
#                 best_run = r

#     if best_run:
#         params = {k: normalize(v) for k, v in best_run.data.params.items() if k in PARAM_KEYS}
#         metrics = best_run.data.metrics
        
#         print(f"\n{'='*70}")
#         print(f"🏆 BEST RUN IDENTIFIED:")
#         print(f"{'='*70}")
#         print(f"   Run Name: {best_run.info.run_name or 'N/A'}")
#         print(f"   Run ID: {best_run.info.run_id}")
#         print(f"   {METRIC_KEY}: {best_metric:.6f}")
#         print(f"   Parameters: {params}")
#         print(f"{'='*70}\n")
        
#         return best_run.info.run_id, params, metrics
#     else:
#         print("⚠ No valid runs with metric found.")
#         return None, {}, {}

# # ================ DUPLICATE VERSION CHECK ===================== #
# def check_duplicate(client, new_params, new_metrics):
#     try:
#         mv_list = client.search_model_versions(f"name = '{REGISTERED_MODEL_NAME}'")
#     except Exception as e:
#         print(f"ℹ No existing model versions found (this may be first registration): {e}")
#         return None
    
#     if not mv_list:
#         return None

#     new_metric_val = new_metrics.get(METRIC_KEY, None)
    
#     for mv in mv_list:
#         try:
#             run = client.get_run(mv.run_id)
#         except Exception as e:
#             print(f"⚠ Could not fetch run {mv.run_id}: {e}")
#             continue

#         old_params = {k: normalize(v) for k, v in run.data.params.items() if k in new_params}
#         old_metric_val = run.data.metrics.get(METRIC_KEY, None)

#         same_params = all(old_params.get(k) == new_params.get(k) for k in new_params)
#         same_metric = (
#             old_metric_val is not None and new_metric_val is not None
#             and abs(old_metric_val - new_metric_val) <= TOL
#         )
        
#         if same_params and same_metric:
#             print(f"\n⏭️ DUPLICATE DETECTED!")
#             print(f"   Existing Version: {mv.version}")
#             print(f"   Run ID: {mv.run_id}")
#             print(f"   This model is already registered with same params & performance.")
#             return mv
            
#     return None

# # ================== REGISTER MODEL LOGIC ======================= #
# def register_model(client, run_id, params, metrics):
#     duplicate_version = check_duplicate(client, params, metrics)
#     if duplicate_version:
#         print(f"✅ Using existing registered version: {duplicate_version.version}")
#         send_slack_notification(f"⚠️ Duplicate detected — using existing model version *v{duplicate_version.version}* for `{REGISTERED_MODEL_NAME}`.")
#         return duplicate_version

#     model_uri = f"runs:/{run_id}/{ARTIFACT_PATH}"
#     print(f"\n⏳ Registering new model version...")
#     print(f"   Model URI: {model_uri}")
#     print(f"   Target: {REGISTERED_MODEL_NAME}")

#     try:
#         new_version = mlflow.register_model(model_uri, REGISTERED_MODEL_NAME)
        
#         print(f"\n{'='*70}")
#         print("✅ MODEL REGISTERED SUCCESSFULLY!")
#         print(f"{'='*70}")
#         print(f"   Model Name: {REGISTERED_MODEL_NAME}")
#         print(f"   Version: {new_version.version}")
#         print(f"   Source Run ID: {run_id}")
#         print(f"   {METRIC_KEY}: {metrics.get(METRIC_KEY, 'N/A')}")
#         print(f"{'='*70}\n")

#         # 🏷️ Add helpful tags
#         client.set_model_version_tag(
#             REGISTERED_MODEL_NAME, 
#             new_version.version, 
#             "source_run_id", 
#             run_id
#         )
#         client.set_model_version_tag(
#             REGISTERED_MODEL_NAME, 
#             new_version.version, 
#             "experiment_name", 
#             EXPERIMENT_NAME
#         )
#         client.set_model_version_tag(
#             REGISTERED_MODEL_NAME, 
#             new_version.version, 
#             "metric_rmse", 
#             str(metrics.get(METRIC_KEY, ""))
#         )
        
#         for param_key, param_val in params.items():
#             client.set_model_version_tag(
#                 REGISTERED_MODEL_NAME,
#                 new_version.version,
#                 f"param_{param_key}",
#                 str(param_val)
#             )

#         # ✅ Send Slack success message
#         send_slack_notification(
#             f"✅ Model *{REGISTERED_MODEL_NAME}* registered successfully as version *v{new_version.version}*.\n"
#             f"🏷 Metric `{METRIC_KEY}` = {metrics.get(METRIC_KEY, 'N/A')}\n"
#             f"🔗 Run ID: {run_id}"
#         )

#         return new_version
        
#     except Exception as e:
#         print(f"❌ Registration Failed: {e}")
#         send_slack_notification(f"❌ Model registration failed for `{REGISTERED_MODEL_NAME}` — {e}")
#         import traceback
#         traceback.print_exc()
#         sys.exit(1)

# # ============================ MAIN ============================ #
# if __name__ == "__main__":
#     print("\n" + "=" * 70)
#     print("🚀 MODEL REGISTRATION - BEST RUN SELECTION (MLflow + UC)")
#     print("=" * 70 + "\n")

#     client = MlflowClient()
    
#     print(f"📋 Configuration:")
#     print(f"   Experiment: {EXPERIMENT_NAME}")
#     print(f"   Target Model: {REGISTERED_MODEL_NAME}")
#     print(f"   Metric to optimize: {METRIC_KEY} (lower is better)")
#     print(f"   Artifact Path: {ARTIFACT_PATH}\n")
    
#     print("🔍 Searching for best run...")
#     run_id, params, metrics = get_best_run(client)

#     if run_id:
#         register_model(client, run_id, params, metrics)
#         print("\n✨ Registration process completed successfully!")
#     else:
#         print("❌ No valid best run found. Exiting.")
#         send_slack_notification(f"⚠️ Model registration skipped — no valid best run found for `{REGISTERED_MODEL_NAME}`.")
#         sys.exit(1)
